In [154]:
import re
import time
import json
import requests
import pandas as pd

from pprint import pprint
from bs4 import BeautifulSoup

In [155]:
headers = {
'Host' : 'auto.ru',
'User-Agent' : 'Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0',
'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
'Accept-Language' : 'en-US,en;q=0.5',
'Accept-Encoding' : 'gzip, deflate, br',
'Connection' : 'keep-alive',
'Cookie' : 'autoru_gdpr=1; suid=bc8be712d638a08131a6e9e0d202d740.c5b54afb905b8c06122ab3a791bcd52b; yandexuid=2760879651631376416; my=YysBAgA%3D; counter_ga_all7=2; credit_filter_promo_popup_closed=true; bltsr=1; autoru-visits-count=2; _csrf_token=cd33b362bade8bf97053fcccb6e8a730d67cea3890bc13dd; from_lifetime=1636905126761; from=direct; autoru_sid=a%3Ag619130a126cnq19ulaaft362m5p2vb2.ce95d73e09b71591f2b431378fbd3c26%7C1636905121806.604800.JcGnv8AgAv4Xvv1e8UHmiQ.qe2Ojz2DfSD_52bJZeYXuKaARhOYKurZFIDQMePmObs; autoruuid=g619130a126cnq19ulaaft362m5p2vb2.ce95d73e09b71591f2b431378fbd3c26; X-Vertis-DC=sas; yuidlt=1; crookie=uhdn1VGOHoS1oc8l4sXp2M5D/rZ7bF0rzNm+BarwG5j6c314PffzGRMzarsbN0UdUvjdLE2fxvOarzvBJ/iU56YxYuU=; cmtchd=MTYzNjkwNTEyNDY5Mw==; deal_million_popup_page_seen=0; los=1',
'Upgrade-Insecure-Requests' : '1',
'Sec-Fetch-Dest' : 'document',
'Sec-Fetch-Mode' : 'navigate',
'Sec-Fetch-Site' : 'none',
'Sec-Fetch-User' : '?1'
}
url = 'https://auto.ru/cars/all/'
page_suffix = '?page='

In [156]:
perks = ['Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня', 'bodyType', 'brand', 'color', 'complectation_dict', 'description', 'engineDisplacement', 
      'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name', 
      'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency', 'productionDate', 'sell_id', 'super_gen', 
      'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'car_url']

In [169]:
def get_ad_info(url, headers, characteristics):
    ad_info = dict()
    ad_id = re.search('[a-z0-9]{8,12}-[a-z0-9]{6,10}', link).group(0)
    r = requests.get(url, headers=headers)
    html = BeautifulSoup(r.content, 'html.parser')
    for li in html.find_all('li', 'CardInfoRow'):
        var1, var2 = tuple(li.children)
        key = var1.string
        value = var2.string
        if key in perks[:7]:
            ad_info[key] = value

    j1 = json.loads(html.script.string)
    j2 = json.loads(html.find('script', id="initial-state").string)
    for k in set(perks).intersection(set(j1)):
        ad_info[k] = j1[k]

    for l in set(perks).intersection(j1['vehicleEngine']):
        ad_info[l] = j1['vehicleEngine'][l]

    for m in set(perks).intersection(j2['card']['state']):
        ad_info[m] = j2['card']['state'][m]

    ad_info['equipment_dict'] = j2['card']['vehicle_info']['equipment']
    ad_info['complectation_dict'] = j2['card']['vehicle_info']['complectation']
    ad_info['model_info'] = j2['card']['vehicle_info']['model_info']
    ad_info['model_name'] = j2['card']['vehicle_info']['model_info']['name']
    ad_info['super_gen'] = j2['card']['vehicle_info']['super_gen']
    ad_info['vendor'] = j2['card']['vehicle_info']['vendor']
    ad_info['sell_id'] = j2['card']['saleId']



    ad_info['price'] = j1['offers']['price']
    ad_info['priceCurrency'] = j1['offers']['priceCurrency']
    ad_info['car_url'] = j1['offers']['url']

    ad_info['parsing_unixtime'] = time.time()

    return ad_info

In [170]:
df = pd.DataFrame(columns=perks)
for page_num in range(1, 2):
    if page_num <=1:
        page_url = url
    else:
        page_url = url + page_suffix + str(page_num)
    print(f'{page_url=}')
    res = requests.get(page_url, headers=headers)
    code = BeautifulSoup(res.content, 'html.parser')
    ad_links = code.find_all("a", class_="ListingItemTitle__link")
    n = len(ad_links)
    for l in ad_links:
        print(n, end=' ')
        n -= 1
        link = l['href']
        car_info = get_ad_info(link, headers, perks)
        df = df.append(car_info, ignore_index=True,)
        time.sleep(1)
'''            
    except Exception as err_msg:
        print(err_msg)
    else:
        print('done smoothly')
    finally:
        print('moving to page', page_num + 1)
'''

page_url='https://auto.ru/cars/all/'
38 37 36 35 34 33 32 31 30 29 28 27 26 25 

KeyError: 'price'

In [159]:
df

,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,bodyType,brand,color,...,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,car_url,price
0,1 владелец,10 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,внедорожник 5 дв.,JEEP,серебристый,...,1.637186e+09,RUB,2018,1105476150-384cb422,"{'id': '20040921', 'name': 'IV (WK2) Рестайлин...",ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,AMERICAN,https://auto.ru/cars/used/sale/jeep/grand_cher...,4000000.0


In [17]:
df.engineDisplacement.value_counts()

Series([], Name: engineDisplacement, dtype: int64)